# Persistence time of a general epidemic

The stochastic general epidemic model with counts $(S, I, R)$ of the number of people susceptible, infectious, and recovered, has events:

* Birth: $(S, I, R) \to (S + 1, I, R)$ with hazard $\mu N$;
* Death of a susceptible: $(S, I, R) \to (S - 1, I, R)$ with hazard $\mu S$;
* Death of an infectious: $(S, I, R) \to (S, I - 1, R)$ with hazard $\mu I$;
* Death of a recovered: $(S, I, R) \to (S, I, R - 1)$ with hazard $\mu R$;
* Infection: $(S, I, R) \to (S - 1, I + 1, R)$ with hazard $\frac{\beta I}{N} S$; and
* Recovery: $(S, I, R) \to (S, I - 1, R + 1)$ with hazard $\gamma I$;

where $N = S + I + R$.

The basic reproduction number is
$$R_0 = \frac{\beta}{\gamma + \mu}.$$

We will use the endemic equilibrium of the deterministic model
$$\begin{aligned}
\frac{\mathrm{d} S}{\mathrm{d} t} &= \mu N - \frac{\beta I}{N} S - \mu S,\\
\frac{\mathrm{d} I}{\mathrm{d} t} &= \frac{\beta I}{N} S - \gamma I - \mu I,\\
\frac{\mathrm{d} R}{\mathrm{d} t} &= \gamma I - \mu R,
\end{aligned}$$
as the expected value of a multinomial distribution to determine the initial condition for the stochastic model:
$$\left(S(0), I(0), R(0)\right)
= \mathrm{Multinomial}(N_0, p),$$
where
$$p = \left(\frac{1}{R_0},
            \frac{\mu}{\beta}\left(R_0 - 1\right),
            \frac{\gamma}{\beta}\left(R_0 - 1\right)\right),$$
and $N_0$ is the initial population size.
We will resample from this multinomial we get an initial condition with $I(0) > 0$.

We will simulate the stochastic model using the Gillespie algorithm.

For each simulation, we will find the extinction time, the first time with $I(t) = 0$.

In [10]:
from matplotlib import pyplot
import numpy
import pandas
from scipy import stats
import seaborn

import sensitivity_analysis


numpy.random.seed(1)


def get_R_0(beta, gamma, mu, N):
    return beta / (gamma + mu)
    

def initial_state(beta, gamma, mu, N_0):
    '''Generate a random initial `state`.'''
    R_0 = get_R_0(beta, gamma, mu, N_0)
    assert R_0 > 1
    p = pandas.Series({'S': 1 / R_0,
                       'I': mu / beta * (R_0 - 1),
                       'R': gamma / beta * (R_0 - 1)})
    # Resample to get a `state` with I > 0.
    while True:
        state = pandas.Series(numpy.random.multinomial(N_0, p),
                              index=p.index)
        if state['I'] > 0:
            break
    return state


# The transitions that can occur and how they change `state`.
transitions = {
    'birth': pandas.Series(dict(S=+1, I=0, R=0)),
    'death of S': pandas.Series(dict(S=-1, I=0, R=0)),
    'death of I': pandas.Series(dict(S=0, I=-1, R=0)),
    'death of R': pandas.Series(dict(S=0, I=0, R=-1)),
    'infection': pandas.Series(dict(S=-1, I=+1, R=0)),
    'recovery': pandas.Series(dict(S=0, I=-1, R=+1)),
}


def update_hazards(hazards, t, state, beta, gamma, mu):
    '''Update `hazards` for the current state and time.'''
    N = state.sum()
    hazards['birth'] = mu * N
    hazards['death of S'] = mu * state['S']
    hazards['death of I'] = mu * state['I']
    hazards['death of R'] = mu * state['R']
    hazards['infection'] = beta * state['I'] / N * state['S']
    hazards['recovery'] = gamma * state['I']


def stop(t, state):
    '''The stopping condition for the simulation.'''
    return (state['I'] == 0)


def persistence_time(beta, gamma, mu, N_0=1000):
    '''Simulate the persistence time for a stochastic general SIR model.'''
    t = 0
    state = initial_state(beta, gamma, mu, N_0)
    # Build empty vectors that will get updated in each step.
    hazards = pandas.Series(index=transitions.keys(),
                            dtype=float)
    hazards_scaled = pandas.Series(index=hazards.index,
                                   dtype=float)
    while not stop(t, state):
        update_hazards(hazards, t, state, beta, gamma, mu)
        # Find the time to the next event.
        hazard_total = hazards.sum()
        t += numpy.random.exponential(1 / hazard_total)
        # Find which of the events occurred.
        # Scale the hazards so that they sum to 1.
        hazards_scaled[:] = hazards / hazard_total
        which = numpy.random.choice(hazards.index, p=hazards_scaled)
        state += transitions[which]
    return t


def gamma_from_mean_and_shape(mean, shape):
    '''Generate a gamma random variable with the given mean and shape.'''
    scale = mean / shape
    return stats.gamma(shape, scale=scale)


transmission_rate = gamma_from_mean_and_shape(0.03, 4)
recovery_rate = gamma_from_mean_and_shape(0.01, 4)
death_rate = gamma_from_mean_and_shape(0.001, 4)
parameters = dict(beta=transmission_rate,
                  gamma=recovery_rate,
                  mu=death_rate)

In [11]:
persistence_time(0.012, 0.01, 0.001)

2269.016766437005

We need to build a function that runs `persistence_time()` for a each parameter set in a sequence of parameter sets.